In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import polars as pl
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

In [3]:
cie_9 = pl.read_excel(
    "../data/external/Esquema_Registro-2023.xlsx",
    sheet_name="Anexo 2",
    read_csv_options={"skip_rows": 5},
)

cie_10 = pl.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")

## 1. Análisis Sociodemográfico de Pacientes Atendidos en el Hospital del Tórax

En este cuadernillo se quiere ver el perfil sociodemográfico de los pacientes hospitalizados
en el Hospital del Tórax. Se utilizará la base de datos pública de egresos hospitalarios DEIS
entre 2001 y 2020. Las preguntas que se quieren responder son:

- ¿Cómo es la distribución geográfica de los pacientes atendidos en el INT?
- ¿Cuál es el rango etario más atendido en el INT?
- ¿Cuál es el género más atendido en el INT?
- ¿Cuál es la previsión más atendida en el INT?
- ¿Cuál es la letra de FONASA más atendida en el INT?

Para responder estas preguntas se realizará una agrupación de los datos con todas las variables
sociodemográficas de interés para el análisis.

In [4]:
# Define todas las variables que se quieren saber

VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS = ["ANO_EGRESO", "REGION_RESIDENCIA", "COMUNA_RESIDENCIA"]
VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS = ["SEXO", "EDAD_CATEGORIA", "PREVISION", "BENEFICIARIO"]
VARIABLE_A_CONTAR = "DIAG1"

agrupacion_total = (
    VARIABLES_SOCIODEMOGRAFICAS_ESTATICAS
    + VARIABLES_SOCIODEMOGRAFICAS_DINAMICAS
    + [VARIABLE_A_CONTAR]
)

Luego, se genera la consulta de la base de datos. Para reducir el coste computacional, se filtra
la base de datos únicamente para los datos del INT. Posteriormente, se agrupa la base de datos
con las variables sociodemográficas de interés y se cuenta la ocurrencia de cada grupo.

In [5]:
desglose_sociodemografico = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .group_by(agrupacion_total)
    .agg([pl.count(), pl.col("INTERV_Q").sum().alias("n_int_q")])
    .collect(streaming=True)
    .sort(["ANO_EGRESO", "count"], descending=True)
)

In [6]:
display(desglose_sociodemografico.sum())

ANO_EGRESO,REGION_RESIDENCIA,COMUNA_RESIDENCIA,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,n_int_q
i64,i64,i64,str,str,str,str,str,u32,i64
148543932,843571,855600363,null,null,null,null,null,96742,24101


Los resultados indican que en todo el periodo analizado el INT presentó 96742 Egresos Hospitalarios
(columna "count") y 24101 Egresos con Procedimientos Quirúrgicos. Esto está en concordancia con el
cuadernillo de egresos hospitalarios.

In [7]:
desglose_sociodemografico_cie = (
    desglose_sociodemografico.join(
        cie_9, how="left", left_on="COMUNA_RESIDENCIA", right_on="Código Comuna"
    )
    .with_columns(pl.col(["PREVISION", "Nombre Comuna"]).str.to_titlecase())
    .drop("Código Región")
)

desglose_sociodemografico_cie = desglose_sociodemografico_cie.join(
    cie_10, how="left", left_on="DIAG1", right_on="Código"
)

desglose_sociodemografico_cie = desglose_sociodemografico_cie.rename(
    {"REGION_RESIDENCIA": "codregion", "COMUNA_RESIDENCIA": "cod_comuna"}
)

In [8]:
display(desglose_sociodemografico_cie.head())

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,n_int_q,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,i64,str,str,str,str,str,str
2020,5,5701,"""Hombre""","""(50, 60]""","""Fonasa""","""D""","""Z511""",8,0,"""San Felipe""","""de Valparaíso""","""Sesión de quim…","""Z51 OTRA ATENC…","""Z40-Z54 CONTA…","""Cap.21 FACTOR…"
2020,13,13122,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""Z511""",8,0,"""Peñalolén""","""Metropolitana …","""Sesión de quim…","""Z51 OTRA ATENC…","""Z40-Z54 CONTA…","""Cap.21 FACTOR…"
2020,7,7101,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""Z511""",8,0,"""Talca""","""del Maule""","""Sesión de quim…","""Z51 OTRA ATENC…","""Z40-Z54 CONTA…","""Cap.21 FACTOR…"
2020,3,3101,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""D381""",6,0,"""Copiapo""","""de Atacama""","""Tumor de compo…","""D38 TUMOR DE C…","""D37-D48 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13106,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""Z291""",6,0,"""Estación Centr…","""Metropolitana …","""Inmunoterapia …","""Z29 NECESIDAD …","""Z20-Z29 PERSO…","""Cap.21 FACTOR…"


En los resultados se indica el perfil sociodemográfico para cada combinación de variable presente
en la base de datos. A modo de ejemplo, hubo 8 egresos en el 2020 de una paciente de la Región
número 13 y Comuna 13122 que tenía entre 60 a 70 años que tenía PREVISION FONASA Letra B diagnosticada
con el diagnóstico Z511.

Uno de los diagnósticos de interés para el Instituto es el de código "C341".

In [9]:
display(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341"))
print(desglose_sociodemografico_cie.filter(pl.col("DIAG1") == "C341").select(pl.col("count").sum()))

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,n_int_q,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
i64,i64,i64,str,str,str,str,str,u32,i64,str,str,str,str,str,str
2020,13,13122,"""Hombre""","""(70, 80]""","""Fonasa""","""B""","""C341""",3,0,"""Peñalolén""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13120,"""Mujer""","""(70, 80]""","""Desconocido""",null,"""C341""",3,3,"""Ñuñoa""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13120,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",2,1,"""Ñuñoa""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,5,5705,"""Mujer""","""(50, 60]""","""Fonasa""","""B""","""C341""",2,0,"""Putaendo""","""de Valparaíso""","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13122,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",2,2,"""Peñalolén""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13123,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",2,1,"""Providencia""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13101,"""Mujer""","""(70, 80]""","""Fonasa""","""B""","""C341""",2,0,"""Santiago""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,11,11101,"""Mujer""","""(60, 70]""","""Fonasa""","""B""","""C341""",2,0,"""Coihaique""","""de Aisén del G…","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"
2020,13,13120,"""Hombre""","""(60, 70]""","""Desconocido""",null,"""C341""",2,0,"""Ñuñoa""","""Metropolitana …","""Tumor maligno …","""C34 TUMOR MALI…","""C30-C39 NEOPL…","""Cap.02 NEOPLA…"


shape: (1, 1)
┌───────┐
│ count │
│ ---   │
│ u32   │
╞═══════╡
│ 1709  │
└───────┘


Por lo tanto, se muestra cada desglose sociodemográfico de los pacientes atendidos. Además,
se indica que la suma de cada conteo de grupo suma 1709 entre 2001 y 2020. Esto está en concordancia
con los egresos totales (1709) para el diagnóstico mencionado en el periodo mencionado.

In [10]:
display(desglose_sociodemografico_cie.null_count())

ANO_EGRESO,codregion,cod_comuna,SEXO,EDAD_CATEGORIA,PREVISION,BENEFICIARIO,DIAG1,count,n_int_q,Nombre Comuna,Nombre Región,Descripción,Categoría,Sección,Capítulo
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,3766,0,0,0,0,0,5,5,5,5


Se muestra que la unión con los diccionarios CIE se realizó correctamente. Solamente 5 registros
carecen de una información CIE sobre el diagnóstico. Por otro lado, todos los registros tienen
información sobre la comuna y región.

In [11]:
desglose_sociodemografico_cie.write_csv(
    "../data/interim/sociodemografico/desglose_sociodemografico.csv",
    separator=";",
)

## Obtención de desglose sociodemográfico para Fibrosis Quística

En el proyecto EPH del INT se tiene alto interés sobre los egresos hospitalarios de Fibrosis Quística.

In [12]:
# Obtiene solo los diagnosticos que tengan 'fibrosis qu'
resumen_fibrosis = desglose_sociodemografico_cie.filter(
    pl.col("Descripción").str.to_lowercase().str.contains("fibrosis qu")
)

# Obtiene los codigos de fibrosis quistica
codigos_fibrosis = resumen_fibrosis.select(pl.col("DIAG1").unique())
nombre_fibrosis = resumen_fibrosis.select(pl.col("Descripción").unique())

print(f"Los codigos para la fibrosis quistica son: {codigos_fibrosis} - {nombre_fibrosis}")

Los codigos para la fibrosis quistica son: shape: (3, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ E848  │
│ E840  │
│ E849  │
└───────┘ - shape: (3, 1)
┌───────────────────────────────────┐
│ Descripción                       │
│ ---                               │
│ str                               │
╞═══════════════════════════════════╡
│ Fibrosis quística con manifestac… │
│ Fibrosis quística con otras mani… │
│ Fibrosis quística, sin otra espe… │
└───────────────────────────────────┘


In [13]:
# Obtiene tabla dinamica de resumen fibrosis
resumen_tabla_fibrosis = pd.pivot_table(
    resumen_fibrosis.to_pandas().query("ANO_EGRESO >= 2018"),
    values="count",
    index="EDAD_CATEGORIA",
    columns="ANO_EGRESO",
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

display(resumen_tabla_fibrosis)

# Exporta el resumen
resumen_tabla_fibrosis.to_excel("../data/interim/resumen_fibrosis_quistica_hospitalizado_int.xlsx")

ANO_EGRESO,2018,2019,2020,All
EDAD_CATEGORIA,,,,
"(10, 20]",20,17,7,44
"(20, 30]",26,23,14,63
"(30, 40]",8,9,7,24
"(40, 50]",8,2,0,10
"(50, 60]",1,1,1,3
"(60, 70]",0,1,1,2
All,63,53,30,146


## Resumen Nacional de Fibrosis Quística

In [14]:
CODIGOS_FIBROSIS = ["E84X", "E840", "E841", "E848", "E849", "P75X"]

In [32]:
# Obtiene Registros de Fibrosis Quistica
nacional_fibrosis_quistica = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("DIAG1").is_in(CODIGOS_FIBROSIS))
    .with_columns(EDAD_CATEGORIA=pl.col("EDAD_A_OS").cut([15, 20, 45, 65, 80], left_closed=True))
    .collect(streaming=True)
)

In [73]:
ORDEN_CATEGORIAS = [
    "[-inf, 15)",
    "[15, 20)",
    "[20, 45)",
    "[45, 65)",
    "[65, 80)",
    "[80, inf)",
]

In [74]:
# Obtiene conteo de egresos para cada diagnostico
resumen_egresos_fibrosis_quistica = (
    nacional_fibrosis_quistica.filter(
        (pl.col("EDAD_CATEGORIA") != "[-inf, 15)") & (pl.col("ANO_EGRESO") >= 2018)
    )
    .group_by(pl.col(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "EDAD_CATEGORIA"]))
    .agg(n_egresos=pl.col("DIAG1").count())
    .to_pandas()
)

# Reordena Categorias
resumen_egresos_fibrosis_quistica["EDAD_CATEGORIA"] = resumen_egresos_fibrosis_quistica[
    "EDAD_CATEGORIA"
].cat.reorder_categories(ORDEN_CATEGORIAS)

In [80]:
# Obtiene cantidad de pacientes de fibrosis nacionales
resumen_n_pacientes_fibrosis_quistica = (
    nacional_fibrosis_quistica.filter(
        (pl.col("EDAD_CATEGORIA") != "[-inf, 15)") & (pl.col("ANO_EGRESO") >= 2018)
    )
    .group_by(pl.col(["ANO_EGRESO", "EDAD_CATEGORIA"]))
    .agg(n_pacientes=pl.col("ID_PACIENTE").n_unique())
    .to_pandas()
)

# Reordena Categorias
resumen_n_pacientes_fibrosis_quistica["EDAD_CATEGORIA"] = resumen_n_pacientes_fibrosis_quistica[
    "EDAD_CATEGORIA"
].cat.reorder_categories(ORDEN_CATEGORIAS)

## Obtener reusmen de egresos Nacionales e INT

In [59]:
# Obtiene tabla dinamica de resumen nacional por grupo etario
tabla_egresos_nacional_fibrosis = pd.pivot_table(
    resumen_egresos_fibrosis_quistica,
    index=["EDAD_CATEGORIA"],
    columns=["ANO_EGRESO"],
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

display(tabla_egresos_nacional_fibrosis)

# Obtiene tabla dinamica de resumen torax por grupo etario
tabla_egresos_torax_fibrosis = pd.pivot_table(
    resumen_egresos_fibrosis_quistica.query("ESTABLECIMIENTO_SALUD == 112103"),
    index=["EDAD_CATEGORIA"],
    columns=["ANO_EGRESO"],
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

display(tabla_egresos_torax_fibrosis)

n_egresos                 
ANO_EGRESO          2018 2019 2020    All
EDAD_CATEGORIA                           
[-inf, 15)             0    0    0    NaN
[15, 20)              83   76   66  225.0
[20, 45)              81   75   63  219.0
[45, 65)              10    7   10   27.0
[65, 80)               6    8    6   20.0
[80, inf)              8    6    4   18.0
All                  188  172  149  509.0

n_egresos                 
ANO_EGRESO          2018 2019 2020    All
EDAD_CATEGORIA                           
[-inf, 15)             0    0    0    NaN
[15, 20)              16   16    5   37.0
[20, 45)              46   35   23  104.0
[45, 65)               1    1    2    4.0
[65, 80)               0    1    0    1.0
[80, inf)              0    0    0    NaN
All                   63   53   30  146.0